In [1]:
# import the libraries as shown below
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings as w
w.filterwarnings('ignore')

from keras.applications.vgg16 import VGG16

In [2]:
DIRECTORY = r'/Users/priya/Desktop/MYLAPIDOCS/Tech/KrishNaik/LungDiseaseDL/chest_xray/train'
CATEGORIES = ["NORMAL", "PNEUMONIA"]

In [3]:
import os
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        if(img != ".DS_Store"):
            image = load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)
            data.append(image)
            labels.append(category)
            

In [4]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)


In [5]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(2, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [10]:
# tell the model what cost and optimization method to use
INIT_LR = 1e-4
EPOCHS = 10
BS = 32
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(
  loss='binary_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
                                  

#test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow(trainX, trainY,
                                                 batch_size = 32)
                                    

In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=(testX, testY),
  epochs=5,
  batch_size=32,
  steps_per_epoch=len(trainX) // BS,
  validation_steps=len(testX) // BS
)

Epoch 1/5
 35/130 [=======>......................] - ETA: 14:40 - loss: 0.6039 - accuracy: 0.7209

KeyboardInterrupt: 

In [ ]:
# loss
import matplotlib.pyplot as plt

plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


In [15]:
mv = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in mv.layers:
    layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(mv.output)

prediction = Dense(2, activation='softmax')(x)

# create a model object
model1 = Model(inputs=mv.input, outputs=prediction)

model1.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [16]:
INIT_LR = 1e-4
EPOCHS = 5
BS = 32
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model1.compile(
  loss='binary_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [17]:
# fit the model
# Run the cell. It will take some time to execute
r1 = model1.fit(
  training_set,
  validation_data=(testX, testY),
  epochs=5,
  batch_size=32,
  steps_per_epoch=len(trainX) // BS,
  validation_steps=len(testX) // BS
)

Epoch 1/5
130/130 [==============================] - 184s 1s/step - loss: 0.5659 - accuracy: 0.7382 - val_loss: 0.8066 - val_accuracy: 0.6445
Epoch 2/5
130/130 [==============================] - 202s 2s/step - loss: 0.4487 - accuracy: 0.7903 - val_loss: 0.8280 - val_accuracy: 0.7285
Epoch 3/5
130/130 [==============================] - 179s 1s/step - loss: 0.4003 - accuracy: 0.8121 - val_loss: 0.9821 - val_accuracy: 0.7373
Epoch 4/5
130/130 [==============================] - 195s 1s/step - loss: 0.3682 - accuracy: 0.8292 - val_loss: 1.3161 - val_accuracy: 0.7393
Epoch 5/5
130/130 [==============================] - 76030s 585s/step - loss: 0.3402 - accuracy: 0.8437 - val_loss: 1.4608 - val_accuracy: 0.7393
